In [1]:
# Data Cleaning and Merging Workflow
# This notebook merges SOURCE_client_data.csv and SOURCE_price_data.csv 
# to create a machine learning-ready dataset

import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("Data Cleaning and Merging Workflow")
print("=" * 50)

# Load the source datasets
print("\n1. Loading Source Datasets")
print("-" * 30)

# Load client data
client_df = pd.read_csv('SOURCE_client_data.csv')
print(f"Client data shape: {client_df.shape}")
print(f"Client data columns: {list(client_df.columns)}")

# Load price data  
price_df = pd.read_csv('SOURCE_price_data.csv')
print(f"Price data shape: {price_df.shape}")
print(f"Price data columns: {list(price_df.columns)}")

# Display first few rows to understand the data structure
print("\nClient data sample:")
print(client_df.head(2))
print("\nPrice data sample:")
print(price_df.head(2))

# Check data types
print("\nClient data types:")
print(client_df.dtypes)
print("\nPrice data types:")
print(price_df.dtypes)

print("\n2. Data Exploration and Understanding")
print("-" * 40)

# Check for missing values and data types
print("Client data info:")
print(client_df.info())
print("\nPrice data info:")
print(price_df.info())

# Check unique values in key columns
print(f"\nUnique client IDs: {client_df['id'].nunique()}")
print(f"Total client records: {len(client_df)}")
print(f"Unique price IDs: {price_df['id'].nunique()}")
print(f"Total price records: {len(price_df)}")

# Check for churn column in client data
print(f"\nChecking for target variable:")
if 'churn' in client_df.columns:
    print("✓ Found 'churn' column in client data")
    print(f"Churn distribution: {client_df['churn'].value_counts()}")
    print(f"Churn rate: {client_df['churn'].mean():.3f}")
else:
    print("✗ No 'churn' column found in client data")

print("\n3. Date Column Processing")
print("-" * 30)

# Identify actual date columns that exist in the data
date_columns_client = [col for col in client_df.columns if 'date' in col.lower()]
date_columns_price = [col for col in price_df.columns if 'date' in col.lower()]

print(f"Date columns in client data: {date_columns_client}")
print(f"Date columns in price data: {date_columns_price}")

def convert_to_epoch(date_series, date_format='%Y-%m-%d'):
    """
    Convert date strings to normalized epoch time (0-1 scale)
    """
    # Convert to datetime
    dates = pd.to_datetime(date_series, format=date_format, errors='coerce')
    
    # Convert to epoch (seconds since 1970-01-01)
    epoch_times = dates.astype('int64') // 10**9
    
    # Normalize to 0-1 scale
    min_epoch = epoch_times.min()
    max_epoch = epoch_times.max()
    
    if max_epoch == min_epoch:
        return epoch_times * 0  # All same date
    
    normalized = (epoch_times - min_epoch) / (max_epoch - min_epoch)
    
    print(f"Date range: {dates.min()} to {dates.max()}")
    print(f"Epoch range: {min_epoch} to {max_epoch}")
    print(f"Normalized range: {normalized.min():.3f} to {normalized.max():.3f}")
    
    return normalized

# Convert date columns in client data
for col in date_columns_client:
    if col in client_df.columns:
        print(f"\nConverting {col}:")
        client_df[col] = convert_to_epoch(client_df[col])

# Convert date columns in price data
for col in date_columns_price:
    if col in price_df.columns:
        print(f"\nConverting {col}:")
        price_df[f'{col}_epoch'] = convert_to_epoch(price_df[col])

print("\n4. Merging Client and Price Data")
print("-" * 35)

# Merge on client ID
print("Performing left join to keep all clients...")
merged_df = client_df.merge(price_df, on='id', how='left')
print(f"Merged dataset shape: {merged_df.shape}")

# Check for missing price data
if len(date_columns_price) > 0:
    missing_price = merged_df[date_columns_price[0]].isna().sum()
    print(f"Clients without price data: {missing_price}")

print("\n5. Creating Price Statistical Features")
print("-" * 40)

# Identify price columns that actually exist
price_columns = [col for col in price_df.columns if col.startswith('price_') and col != 'price_date']
print(f"Found price columns: {price_columns}")

if price_columns:
    # Group price data by client ID to create statistical features
    print("Calculating price statistics per client...")
    
    # Create aggregation dictionary for existing price columns
    agg_dict = {}
    for col in price_columns:
        agg_dict[col] = ['mean', 'std', 'min', 'max', 'last']
    
    price_stats = price_df.groupby('id').agg(agg_dict).round(6)
    
    # Flatten column names
    price_stats.columns = ['_'.join(col).strip() for col in price_stats.columns]
    price_stats = price_stats.reset_index()
    
    print(f"Price statistics shape: {price_stats.shape}")
    print("Sample price statistics:")
    print(price_stats.head(2))
    
    # Merge price statistics with client data
    final_df = client_df.merge(price_stats, on='id', how='left')
    print(f"Dataset with price features shape: {final_df.shape}")
else:
    print("No price columns found - using client data only")
    final_df = client_df.copy()

print("\n6. One-Hot Encoding Categorical Variables")
print("-" * 42)

# Check for categorical columns that actually exist
categorical_cols = final_df.select_dtypes(include=['object']).columns.tolist()
print(f"Found categorical columns: {categorical_cols}")

for col in categorical_cols:
    if col != 'id':  # Don't encode ID column
        print(f"\nUnique {col} values:")
        print(final_df[col].value_counts())
        
        # One-hot encode the column
        dummies = pd.get_dummies(final_df[col], prefix=col)
        print(f"Created {len(dummies.columns)} dummy variables for {col}:")
        print(list(dummies.columns))
        
        # Add to dataset
        final_df = pd.concat([final_df, dummies], axis=1)

print(f"\nDataset shape after one-hot encoding: {final_df.shape}")

print("\n7. Creating Consumption Features")
print("-" * 35)

# Check for consumption-related columns
consumption_cols = [col for col in final_df.columns if 'cons' in col.lower()]
print(f"Found consumption columns: {consumption_cols}")

# Create consumption features if the required columns exist
if 'cons_12m' in final_df.columns and 'forecast_cons_12m' in final_df.columns:
    final_df['cons_pwr_12_mo_dif'] = final_df['cons_12m'] - final_df['forecast_cons_12m']
    final_df['cons_pwr_12_mo_perc'] = np.where(
        final_df['forecast_cons_12m'] != 0,
        (final_df['cons_12m'] - final_df['forecast_cons_12m']) / final_df['forecast_cons_12m'],
        0
    )
    print("Created consumption difference features")
else:
    print("Required consumption columns not found - skipping consumption features")

print("\n8. Creating Price Change Features")
print("-" * 35)

# Create price change features if price statistics exist
if price_columns:
    price_types = []
    for col in price_columns:
        price_type = col.replace('price_', '')
        price_types.append(price_type)
    
    for price_type in price_types:
        min_col = f'price_{price_type}_min'
        max_col = f'price_{price_type}_max'
        
        if min_col in final_df.columns and max_col in final_df.columns:
            # Price difference (max - min)
            dif_col = f'price_{price_type}_dif'
            final_df[dif_col] = final_df[max_col] - final_df[min_col]
            
            # Price percentage change
            perc_col = f'price_{price_type}_perc'
            final_df[perc_col] = np.where(
                final_df[min_col] != 0,
                (final_df[max_col] - final_df[min_col]) / final_df[min_col],
                0
            )
            
            print(f"Created {dif_col} and {perc_col}")
else:
    print("No price columns found - skipping price change features")

print("\n9. Handle Target Variable")
print("-" * 30)

# Handle churn column based on what actually exists
if 'churn' in final_df.columns:
    print("Using existing churn column")
    # Ensure churn is integer (0/1)
    final_df['churn'] = final_df['churn'].astype(int)
    
    churn_distribution = final_df['churn'].value_counts()
    print("Churn distribution:")
    print(churn_distribution)
    print(f"Churn rate: {final_df['churn'].mean():.3f}")
else:
    print("No churn column found - creating from available data")
    # Try to create from date_end if it exists
    if 'date_end' in final_df.columns:
        # Before normalization, check if date_end indicates churn
        original_client = pd.read_csv('SOURCE_client_data.csv')
        if 'date_end' in original_client.columns:
            final_df['churn'] = original_client['date_end'].notnull().astype(int)
            print("Created churn from date_end column")
        else:
            final_df['churn'] = 0  # Default to no churn
            print("Created default churn column (all 0)")
    else:
        final_df['churn'] = 0  # Default to no churn
        print("Created default churn column (all 0)")

print("\n10. Data Cleaning and Normalization")
print("-" * 35)

def clean_and_normalize_features(df):
    """
    Clean and normalize numerical features for ML readiness
    """
    # Replace infinite values with NaN
    df = df.replace([np.inf, -np.inf], np.nan)
    
    # Get numerical columns (excluding ID, target, and categorical)
    numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    # Remove columns that shouldn't be normalized
    columns_to_exclude = ['id', 'churn']
    numerical_cols = [col for col in numerical_cols if col not in columns_to_exclude]
    
    print(f"Normalizing {len(numerical_cols)} numerical features...")
    
    # Normalize numerical features to 0-1 scale
    for col in numerical_cols:
        if col in df.columns:
            col_min = df[col].min()
            col_max = df[col].max()
            
            if pd.notna(col_min) and pd.notna(col_max) and col_max != col_min:
                df[col] = (df[col] - col_min) / (col_max - col_min)
            else:
                df[col] = df[col].fillna(0)
    
    # Fill any remaining NaN values with 0 (except for churn column)
    for col in df.columns:
        if col != 'churn' and df[col].isnull().any():
            df[col] = df[col].fillna(0)
    
    print("Completed normalization and missing value handling")
    
    return df

final_df = clean_and_normalize_features(final_df)

print("\n11. Remove Non-Numeric Columns")
print("-" * 35)

# Identify non-numeric columns
non_numeric_cols = []
for col in final_df.columns:
    if col not in ['id', 'churn']:  # Keep ID and target for now
        dtype = final_df[col].dtype
        if dtype == 'object' or dtype == 'category':
            non_numeric_cols.append(col)
        elif dtype == 'bool':
            print(f"Converting boolean column '{col}' to int")
            final_df[col] = final_df[col].astype(int)

print(f"Found {len(non_numeric_cols)} non-numeric columns to remove:")
for col in non_numeric_cols:
    print(f"- {col}: {final_df[col].dtype}")

# Drop non-numeric columns
if non_numeric_cols:
    final_df = final_df.drop(columns=non_numeric_cols)
    print(f"Dropped {len(non_numeric_cols)} non-numeric columns")

print("\n12. Final Dataset Preparation")
print("-" * 32)

# Remove ID and any remaining unnecessary columns
columns_to_remove = ['id']
# Add any date columns that were kept
columns_to_remove.extend([col for col in final_df.columns if 'date' in col.lower() and col != 'churn'])

final_df = final_df.drop(columns=[col for col in columns_to_remove if col in final_df.columns])

# Reorder columns - features first, then target
target_col = 'churn'
feature_cols = [col for col in final_df.columns if col != target_col]
final_df = final_df[feature_cols + [target_col]]

print(f"Final dataset shape: {final_df.shape}")
print(f"Features: {len(feature_cols)}")
print(f"Target variable: {target_col}")

print("\n13. Data Quality Checks")
print("-" * 25)

# Check for any remaining issues
print("Data quality checks:")
print(f"- Missing values: {final_df.isnull().sum().sum()}")
print(f"- Infinite values: {np.isinf(final_df.select_dtypes(include=[np.number])).sum().sum()}")
print(f"- Duplicate rows: {final_df.duplicated().sum()}")
print(f"- Data types: {final_df.dtypes.value_counts().to_dict()}")

# Verify all columns are numeric
object_cols = final_df.select_dtypes(include=['object']).columns.tolist()
if object_cols:
    print(f"WARNING: Found remaining object columns: {object_cols}")
else:
    print("✓ All columns are now numeric")

print("\n14. Saving Cleaned Dataset")
print("-" * 30)

# Save the final dataset
output_file = 'DATA_v2_churn.csv'
final_df.to_csv(output_file, index=False)

print(f"Dataset saved as: {output_file}")
print(f"Final shape: {final_df.shape}")

print("\n15. Create Sample Dataset")
print("-" * 30)

# Create a sample with 150 records
sample_df = final_df.sample(n=min(150, len(final_df)), random_state=42)
sample_file = 'SAMPLE_v2_churn.csv'
sample_df.to_csv(sample_file, index=False)

print(f"Sample dataset saved as: {sample_file}")
print(f"Sample shape: {sample_df.shape}")

print("\n" + "="*50)
print("DATA CLEANING AND MERGING COMPLETE!")
print("="*50)

# Display final summary
print("\nFinal column names:")
print(list(final_df.columns))

print("\nFirst 3 rows of cleaned dataset:")
print(final_df.head(10))

print(f"\nDataset ready for machine learning!")
print(f"Full dataset: {output_file}")
print(f"Sample dataset: {sample_file}")

Data Cleaning and Merging Workflow

1. Loading Source Datasets
------------------------------
Client data shape: (14606, 26)
Client data columns: ['id', 'channel_sales', 'cons_12m', 'cons_gas_12m', 'cons_last_month', 'date_activ', 'date_end', 'date_modif_prod', 'date_renewal', 'forecast_cons_12m', 'forecast_cons_year', 'forecast_discount_energy', 'forecast_meter_rent_12m', 'forecast_price_energy_off_peak', 'forecast_price_energy_peak', 'forecast_price_pow_off_peak', 'has_gas', 'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act', 'net_margin', 'num_years_antig', 'origin_up', 'pow_max', 'churn']
Price data shape: (193002, 8)
Price data columns: ['id', 'price_date', 'price_off_peak_var', 'price_peak_var', 'price_mid_peak_var', 'price_off_peak_fix', 'price_peak_fix', 'price_mid_peak_fix']

Client data sample:
                                 id                     channel_sales  \
0  24011ae4ebbe3035111d65fa7c15bc57  foosdfpfkusacimwkcsosbicdxkicaua   
1  d29c2c54acc38f